In [ ]:
# Entendendo a geração de proximos tokens, diferencas de modelos e devices, e lembrando o que foi dito

In [ ]:
# vendo se temos GPU disponivel

# !nvidia-smi <-- não vai funcionar se não tiver GPU

import numpy as np
import torch
import transformers

torch.cuda.is_available() # retorna Ture se encontrou uma GPU para mandar os jobs

# identificando o dispositivo dinamicamente
device = 'cuda' if torch.cuda.is_available() else 'cpu'

print(device)


cuda


In [ ]:
tiny_llama = transformers.pipeline(
    task = "text-generation",
    model="TinyLlama/TinyLlama-1.1B-Chat-v1.0",
    device=device
)

print(tiny_llama)
print(tiny_llama.model)

config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 2048)
    (layers): ModuleList(
      (0-21): 22 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=256, bias=False)
          (v_proj): Linear(in_features=2048, out_features=256, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=2048, out_features=5632, bias=False)
          (up_proj): Linear(in_features=2048, out_features=5632, bias=False)
          (down_proj): Linear(in_features=5632, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
      )
    )
    (norm): 

In [ ]:
prompt = 'today is a beautiful day to'

# Maneira mais direta possivel de usar um modelo. Ele vai usar os parametros padrões para gerar a saida
output = tiny_llama(prompt)

# Se nao especicarmos o maximo de novos tokens ele vai gerar ate encontrar um token que sinalisa fim da sentenca

print(output)


[{'generated_text': 'today is a beautiful day to be alive.\n\n(The camera pans out to show the sun shining brightly, the sky turning from blue to pink, and the leaves rustling in the wind.)\n\nNarrator: And as we look around, we see the world around us in a new light.\n\n(The camera pans out to show the city skyline, the buildings glowing in the sunlight, and the people walking around, smiling and laughing.)\n\nNarrator: We see the beauty of the world around us, and we know that we are lucky to be alive.\n\n(The camera pans out to show the sun setting behind the mountains, the sky turning from pink to orange, and the stars twinkling in the night sky.)\n\nNarrator: And as we watch the sun set, we know that we are witnessing a moment in time that will never be repeated.\n\n(The camera pans out to show the sky turning from orange to red, and the sun setting behind the mountains, the sky turning into a beautiful shade of purple.)\n\nNarrator: We see the beauty of the world around us, and w

In [ ]:
# vendo token a token o processo de geracao

# tensor eh o array que redes neurais utilizam pt --> pytorch | tf --> tensorflow
# só o tensor tem o metodo to(devide), que serve para mover o array para o mesmo dispositivo que o nosso modelo esta

prompt_input_ids = tiny_llama.tokenizer.encode(prompt, return_tensors='pt').input_ids.to(device)

# para inspecionar a geração de tokens, precisamos gerar o output com novos parametros

output = tiny_llama.model.generate(
    prompt_input_ids,
    max_new_tokens = 25,
    return_dict_in_generate = True, # retorne um dicionario com informacoes da geracao de dados
    output_scores = True, # retorne as pontuacoes de cada token gerado
)

print(output.keys())


AttributeError: 'Tensor' object has no attribute 'input_ids'

In [ ]:
# vamos pegar os TRANSITION SCORES (probabildidade da palavra utilizada)

transitions = tiny_llama.model.compute_transition_scores(
    output.sequences,
    output.scores,
    normalize_logits=True
)

In [ ]:
# só quero ver a sequencia de transicao do que foi gerado (ignora o prompt original)

tamanho_prompt = len(prompt_input_ids[0])
print(tamanho_prompt)

# nosso prompt faz parte da resposta gerada
print(output.sequences[0][:tamanho_prompt])


generated_tokens = output.sequences[0][tamanho_prompt:]
print(generated_tokens)

print('| token id | score | token str | pro %')
for (token, score) in zip(generated_tokens,transitions[0]):
    print(token, score.to('cpu').numpy(), tiny_llama.tokenizer.decode(token),np.exp(score.to('cpu').numpy()))

In [ ]:
# Vendo o tempo de execucao para gerar um prompt

start = time.time()

output = tiny_llama(prompt, max_new_tokens=100)

end = time.time()

print(tiny_llama.model)
print(f'Tempo de execucao: {end - start:.2f} segundos')
print(output)
print(output[0]['generated_text'])

In [ ]:
# Lembrando da conversa

prompt1 = "What day is today?"
prompt2 = "What day is tomorrow?"

output = tiny_llama(prompt1, max_new_tokens=10)
print(output[0]['generated_text'])

print("-"*80)

output = tiny_llama(prompt2, max_new_tokens=10)
print(output[0]['generated_text'])

# Ué, o que aconteceu? Alucinou?

In [ ]:
# para se lembrar, precisamos alimentar ele com TUDO QUE FOI GERADO ANTERIORMENTE

prompt1 = "What day is today?"
prompt2 = "What day is tomorrow?"

output = tiny_llama(prompt1, max_new_tokens=10)
print(output[0]['generated_text'])

print("-"*80)

output = tiny_llama(output[0]['generated_text'] + prompt2, max_new_tokens=30)
print(output[0]['generated_text'])

In [ ]:
# vamos carregar outro modelo (alucinou demais anteriormente)

# O código abaixo vai explodir a memória ram (pq ja temos o outro modelo carregado nela).
# deletando e limpando a memória

# import gc # garbage colector
# del tiny_llama
# gc.collect()
# torch.cuda.empty_cache()

# falcon = transformers.pipeline(
#     task='text-generation',
#     model='tiiuae/falcon-7b',
#     device=device
# )